# Magic Method

所有以双下划线包起来的方法，都统称为"魔术方法"。使用这些魔术方法可以构造出优美的代码，将复杂的逻辑封装成简单的方法。

那么一个类中有哪些魔术方法呢？可以使用 Python 内置的方法 `dir()` 来列出类中所有的魔术方法。

In [1]:
#!/usr/bin/env python3
# -*- coding: UTF-8 -*-

class User(object):
    pass

if __name__ == '__main__':
    print(dir(User()))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__']


实际上，创建一个类的过程分为两步，一步是创建类的对象，还有一步就是对类进行初始化。

`__new__` 用来创建类并返回这个类的实例，而`__init__` 只是将传入的参数来初始化该实例。`__new__` 在创建一个实例的过程中必定会被调用，但 `__init__` 就不一定，比如通过 `pickle.load` 的方式反序列化一个实例时就不会调用 `__init__` 方法。

`def __new__(cls)` 是在 `def __init__(self)` 方法之前调用的，作用是返回一个实例对象。此外，`__new__` 方法总是需要返回该类的一个实例，而 `__init__` 不能返回除了 None 的任何值。

In [2]:
#!/usr/bin/env python3
# -*- coding: UTF-8 -*-

class User(object):
    def __new__(cls, *args, **kwargs):
        # 打印 __new__方法中的相关信息
        print('调用了 def __new__ 方法')
        print(args)
        # 最后返回父类的方法
        return super(User, cls).__new__(cls)

    def __init__(self, name, age):
        print('调用了 def __init__ 方法')
        self.name = name
        self.age = age

if __name__ == '__main__':
    usr = User('两点水', 23)

调用了 def __new__ 方法
('两点水', 23)
调用了 def __init__ 方法


在实际开发中，很少会用到 `__new__` 方法，除非你希望能够控制类的创建。通常讲到 `__new__` ，都是牵扯到 metaclass(元类)的。

当然当一个对象的生命周期结束的时候，析构函数 `__del__` 方法会被调用。但是这个方法是 Python 自己对对象进行垃圾回收的。

之前也有讲到过，Python 没有真正意义上的私有属性。然后这就导致了对 Python 类的封装性比较差。我们有时候会希望 Python 能够定义私有属性，然后提供公共可访问的 `get` 方法和 `set` 方法。Python 其实可以通过魔术方法来实现封装。

| 方法  | 说明  |
| --- | --- |
| `__getattr__(self, name)` | 该方法定义了试图访问一个不存在的属性时的行为。重载该方法可以实现捕获错误拼写然后进行重定向，或者对一些废弃的属性进行警告。 |
| `__setattr__(self, name, value)` | 定义了对属性进行赋值和修改操作时的行为。不管对象的某个属性是否存在，都允许为该属性进行赋值。需要注意，实现 `__setattr__` 时要避免"无限递归"的错误。 |
| `__delattr__(self, name)` | `__delattr__` 与 `__setattr__` 很像，只是它定义的是你删除属性时的行为。实现 `__delattr__` 同样要避免"无限递归"的错误。 |
| `__getattribute__(self, name)` | `__getattribute__` 定义了你的属性被访问时的行为，而 `__getattr__` 只有该属性不存在时才会起作用。因此，在支持 `__getattribute__`的 Python 版本，调用`__getattr__` 前必定会调用 `__getattribute__`。使用`__getattribute__` 同样要避免"无限递归"的错误。 |

在进行属性访问控制定义的时候会很容易的引起一个错误：

In [ ]:
# 错误版本
def __setattr__(self, name, value):
    self.name = value
    # 每当属性被赋值的时候， ``__setattr__()`` 会被调用，这样就造成了递归调用。
    # 这意味这会调用 ``self.__setattr__('name', value)`` ，每次方法会调用自己。这样会造成程序崩溃。

# 正确版本
def __setattr__(self, name, value):
    # 给类中的属性名分配值
    self.__dict__[name] = value  
    # 定制特有属性

In [3]:
#!/usr/bin/env python3
# -*- coding: UTF-8 -*-

class User(object):
    def __getattr__(self, name):
        print('调用了 __getattr__ 方法')
        return super(User, self).__getattr__(name)

    def __setattr__(self, name, value):
        print('调用了 __setattr__ 方法')
        return super(User, self).__setattr__(name, value)

    def __delattr__(self, name):
        print('调用了 __delattr__ 方法')
        return super(User, self).__delattr__(name)

    def __getattribute__(self, name):
        print('调用了 __getattribute__ 方法')
        return super(User, self).__getattribute__(name)


if __name__ == '__main__':
    user = User()
    # 设置属性值，会调用 __setattr__
    user.attr1 = True
    # 属性存在,只有__getattribute__调用
    user.attr1
    try:
        # 属性不存在, 先调用__getattribute__, 后调用__getattr__
        user.attr2
    except AttributeError:
        pass
    # __delattr__调用
    del user.attr1


调用了 __setattr__ 方法
调用了 __getattribute__ 方法
调用了 __getattribute__ 方法
调用了 __getattr__ 方法
调用了 __delattr__ 方法


`__get__()`, `__set__()` 和 `__delete__()`，有这些方法的对象叫做描述器。默认对属性的访问控制是从对象的字典里面 (`__dict__`) 中获取 (`get`) , 设置 (`set`) 和删除 (`delete`) 。

自定义容器：

| 功能  | 说明  |
| --- | --- |
| 自定义不可变容器类型 | 需要定义 `__len__` 和 `__getitem__` 方法 |
| 自定义可变类型容器 | 在不可变容器类型的基础上增加定义 `__setitem__` 和 `__delitem__` |
| 自定义的数据类型需要迭代 | 需定义 `__iter__` |
| 返回自定义容器的长度 | 需实现 `__len__(self)` |
| 自定义容器可以调用 `self[key]` ，如果 key 类型错误，抛出TypeError，如果没法返回 key 对应的数值，抛出ValueError | 需要实现 `__getitem__(self, key)` |
| 当执行 `self[key] = value` 时 | 调用是 `__setitem__(self, key, value)`这个方法 |
| 当执行 `del self[key]` 方法 | 其实调用的方法是 `__delitem__(self, key)` |
| 当你想你的容器可以执行 `for x in container:` 或者使用 `iter(container)` 时 | 需要实现 `__iter__(self)` ，该方法返回的是一个迭代器 |

来看一下使用上面魔术方法实现 Haskell 语言中的一个数据结构：

In [4]:
#!/usr/bin/env python3
# -*- coding: UTF-8 -*-

class FunctionalList:
    ''' 实现了内置类型list的功能,并丰富了一些其他方法: head, tail, init, last, drop, take'''

    def __init__(self, values=None):
        if values is None:
            self.values = []
        else:
            self.values = values

    def __len__(self):
        return len(self.values)

    def __getitem__(self, key):
        return self.values[key]

    def __setitem__(self, key, value):
        self.values[key] = value

    def __delitem__(self, key):
        del self.values[key]

    def __iter__(self):
        return iter(self.values)

    def __reversed__(self):
        return FunctionalList(reversed(self.values))

    def append(self, value):
        self.values.append(value)

    def head(self):
        # 获取第一个元素
        return self.values[0]

    def tail(self):
        # 获取第一个元素之后的所有元素
        return self.values[1:]

    def init(self):
        # 获取最后一个元素之前的所有元素
        return self.values[:-1]

    def last(self):
        # 获取最后一个元素
        return self.values[-1]

    def drop(self, n):
        # 获取所有元素，除了前N个
        return self.values[n:]

    def take(self, n):
        # 获取前N个元素
        return self.values[:n]

比较运算符：

| 魔术方法 | 说明  |
| --- | --- |
| `__cmp__(self, other)` | 如果该方法返回负数，说明 `self < other`; 返回正数，说明 `self > other`; 返回 0 说明 `self == other`。强烈不推荐来定义 `__cmp__` , 最好分别定义 `__lt__`, `__eq__` 等方法从而实现比较功能。 `__cmp__` 在 Python3 中被废弃了。 |
| `__eq__(self, other)` | 定义了比较操作符 == 的行为 |
| `__ne__(self, other)` | 定义了比较操作符 != 的行为 |
| `__lt__(self, other)` | 定义了比较操作符 < 的行为 |
| `__gt__(self, other)` | 定义了比较操作符 > 的行为 |
| `__le__(self, other)` | 定义了比较操作符 <= 的行为 |
| `__ge__(self, other)` | 定义了比较操作符 >= 的行为 |

算术运算符：

| 魔术方法 | 说明 |
| --- | --- | 
| `__add__(self, other)` | 实现了加号运算 |
| `__sub__(self, other)` | 实现了减号运算 |
| `__mul__(self, other)` | 实现了乘法运算 |
| `__floordiv__(self, other)` | 实现了 // 运算符 |
| `___div__(self, other)` | 实现了 / 运算符. 该方法在 Python3 中废弃。原因是 Python3 中 division 默认就是 true division |
| `__truediv__(self, other)` | 实现了 true division。只有声明了 `from __future__ import division` 该方法才会生效 |
| `__mod__(self, other)` | 实现了 % 运算符, 取余运算 |
| `__divmod__(self, other)` | 实现了 divmod() 內建函数 |
| `__pow__(self, other)` | 实现了 `**` 次方操作 |
| `__lshift__(self, other)` | 实现了位操作 `<<` |
| `__rshift__(self, other)` | 实现了位操作 `>>` |
| `__and__(self, other)` | 实现了位操作 `&` |
| `__or__(self, other)` | 实现了位操作 ` | ` |
| `__xor__(self, other)` | 实现了位操作 `^` |